## [Design Twitter](https:#leetcode.com/problems/design-twitter/description/)

Medium

Design a simplified version of Twitter where users can post tweets, follow/unfollow another user, and is able to see the 10 most recent tweets in the user's news feed.

Implement the Twitter class:

Twitter(): Initializes your twitter object.   

void postTweet(int userId, int tweetId) Composes a new tweet with ID tweetId by the user userId. Each call to this function will be made with a unique tweetId.   

List<Integer> getNewsFeed(int userId) Retrieves the 10 most recent tweet IDs in the user's news feed. Each item in the news feed must be posted by users who the user followed or by the user themself. Tweets must be ordered from most recent to least recent.   

void follow(int followerId, int followeeId) The user with ID followerId started following the user with ID followeeId.   

void unfollow(int followerId, int followeeId) The user with ID followerId started unfollowing the user with ID followeeId.   
 

Example 1:   

Input   
["Twitter", "postTweet", "getNewsFeed", "follow", "postTweet", "getNewsFeed", "unfollow", "getNewsFeed"]    
[[], [1, 5], [1], [1, 2], [2, 6], [1], [1, 2], [1]]  
Output   
[null, null, [5], null, null, [6, 5], null, [5]]   

Explanation   
Twitter twitter = new Twitter();   
twitter.postTweet(1, 5); # User 1 posts a new tweet (id = 5).   
twitter.getNewsFeed(1);  # User 1's news feed should return a list with 1 tweet id -> [5]. return [5]    
twitter.follow(1, 2);    # User 1 follows user 2.   
twitter.postTweet(2, 6); # User 2 posts a new tweet (id = 6). 
twitter.getNewsFeed(1);  # User 1's news feed should return a list with 2 tweet ids -> [6, 5]. Tweet id 6 should precede tweet id 5 because it is posted after tweet id 5.   
twitter.unfollow(1, 2);  # User 1 unfollows user 2.   
twitter.getNewsFeed(1);  # User 1's news feed should return a list with 1 tweet id -> [5], since user 1 is no longer following user 2.   
 

Constraints:

1 <= userId, followerId, followeeId <= 500   
0 <= tweetId <= 104   
All the tweets have unique IDs.   
At most 3 * 104 calls will be made to postTweet, getNewsFeed, follow, and unfollow.    
A user cannot follow himself.    

In [1]:
from collections import defaultdict
import heapq

class Twitter:

    def __init__(self):
        self.tweetmap = defaultdict(list)
        self.followmap = defaultdict(set)
        self.timestamp = 0

    def postTweet(self, userId: int, tweetId: int) -> None:
        self.tweetmap[userId].append((self.timestamp, tweetId))
        self.timestamp += 1

    def getNewsFeed(self, userId: int) -> list[int]:
        result = []
        self.followmap[userId].add(userId)
        tweetheap = []

        for id in self.followmap[userId]:
            if id in self.tweetmap and self.tweetmap[id]:
                index = len(self.tweetmap[id])-1
                timestamp, tweetId = self.tweetmap[id][index]
                heapq.heappush(tweetheap, (-timestamp, tweetId, id, index-1))

        while tweetheap and len(result) < 10:
            _, rtweetId, ruserId, rindex = heapq.heappop(tweetheap)
            result.append(rtweetId)

            if rindex >= 0:
                nexttimestamp, nexttweetId = self.tweetmap[ruserId][rindex]
                heapq.heappush(tweetheap, (-nexttimestamp, nexttweetId, ruserId, rindex-1))
        
        return result

    def follow(self, followerId: int, followeeId: int) -> None:
        self.followmap[followerId].add(followeeId)

    def unfollow(self, followerId: int, followeeId: int) -> None:
        if followerId != followeeId and followeeId in self.followmap[followerId]:
            self.followmap[followerId].remove(followeeId)


# Your Twitter object will be instantiated and called as such:
# twitter = Twitter();
# twitter.postTweet(1, 5); # User 1 posts a new tweet (id = 5).
# twitter.getNewsFeed(1);  # User 1's news feed should return a list with 1 tweet id -> [5]. return [5]
# twitter.follow(1, 2);    # User 1 follows user 2.
# twitter.postTweet(2, 6); # User 2 posts a new tweet (id = 6).
# twitter.getNewsFeed(1);  # User 1's news feed should return a list with 2 tweet ids -> [6, 5]. Tweet id 6 should precede tweet id 5 because it is posted after tweet id 5.
# twitter.unfollow(1, 2);  # User 1 unfollows user 2.
# twitter.getNewsFeed(1);  # User 1's news feed should return a list with 1 tweet id -> [5], since user 1 is no longer following user 2.



T = Twitter()

T.postTweet(1, 101)  
T.postTweet(2, 201)
T.postTweet(3, 301)

T.follow(1, 2)
T.follow(1, 3)

T.postTweet(2, 202)
T.postTweet(3, 302)
T.postTweet(1, 102)

print(T.getNewsFeed(1))   # big feed (multiple users)

T.follow(1, 4)

T.postTweet(4, 401)
T.postTweet(4, 402)

print(T.getNewsFeed(1))

T.unfollow(1, 3)

print(T.getNewsFeed(1))


[102, 302, 202, 301, 201, 101]
[402, 401, 102, 302, 202, 301, 201, 101]
[402, 401, 102, 202, 201, 101]


**Hint**:
* When merging multiple “latest-first” lists, heaps are a natural way to get global sorted order.
* Push only the **latest tweet** from each user initially.
* After popping from the heap, push that user's next tweet → multi-stream merging.
* Guarantees top 10 are correct even if some users have no recent tweets.
* Follow/unfollow relationships determine whose tweet-lists participate in merging.
* Using timestamps avoids storing actual time; an increasing counter is enough.


**Approach**: Max-Heap Multi-Stream Merge (Top-K Feed Construction)

**Main Logic:**

* For each user, keep:

  * A list of their tweets, each with a timestamp.
  * A set of who they follow.
* When a user requests the feed:

  * Add the user to their own follow set.
  * For every followed user:

    * Take only their **latest tweet** and push it into a max-heap (simulated with negative timestamp).
  * Repeatedly pop from the heap (max timestamp first).
  * Add the chosen tweet to the feed result.
  * If that user has older tweets, push the **next older** tweet from that user into the heap.
  * Continue until you collect 10 tweets or the heap is empty.
* The heap naturally mixes tweets from different users based on timestamp, ensuring the global ordering is correct.

**Key Idea:**

Use a max-heap to merge multiple tweet streams (one per user) and always pick the most recent available tweet.
This is identical to merging multiple sorted lists from newest → oldest.

| Problem              | Design Twitter                                                                     |
| -------------------- | ---------------------------------------------------------------------------------- |
| LeetCode Problem     | 355                                                                                |
| Approach             | **Max-Heap Multi-Stream Merge**                                                    |
| When to apply        | When multiple sources produce time-sorted items and you need recent/top-k results. |
| Clues                | “Get the 10 most recent,” “merge many streams,” “users post at different times.”   |
| Lessons learned      | Use a heap to blend multiple sorted streams by priority (timestamp).               |
| Hidden pattern       | K-way merge using a heap, similar to merging k sorted lists.                       |
| To recognize earlier | When each user has their own sorted list of tweets → classic heap merging problem. |
| Signal words         | “Most recent,” “top 10,” “follow graph,” “sorted by time,” “news feed.”            |